In [1]:
import glob
import netCDF4 as nc
import numpy as np
from rapidfuzz import process
import pandas as pd
import re
import xarray as xr
import geopandas as gpd
from functools import reduce
pd.set_option('display.max_rows', 200)

# crop indices

In [2]:
i= '/Users/caropark/Dropbox (Harvard University)/DataByCountry/Portugal'
end_fao = '/fao_data.nc'
end_cal = '/crop_calendar_data_sage.nc'

fao = nc.Dataset(str(i)+end_fao)
crops = np.transpose(fao['crop']).tobytes()
blob = re.findall('[A-Z][^A-Z]*',  crops.decode('latin-1'))
fao_key = pd.DataFrame({"crop": blob}).reset_index().rename({"index": "fao_idx", "crop": "cropname"}, axis=1)
fao_key = fao_key.applymap(lambda x: x.strip() if isinstance(x, str) else x)

end1 = "/harvest_area_fraction_data_sage_all.nc"
area = xr.open_dataset(str(i)+end1)
crops2 = np.transpose(area['crop_FAO'].values).tobytes()
blob = re.findall('[A-Z][^A-Z]*',  crops2.decode('latin-1'))
area_key = pd.DataFrame({"crop": blob}).reset_index().rename({"index": "harv_area_idx", "crop": "cropname"}, axis=1)
area_key = area_key.applymap(lambda x: x.strip() if isinstance(x, str) else x)

calendar = xr.open_dataset(str(i)+end_cal)
crops2 = np.transpose(xr.open_dataset(str(i)+end_cal)['crop_FAO'].values).tobytes()
blob = re.findall('[A-Z][^A-Z]*',  crops2.decode('latin-1'))
cal_key = pd.DataFrame({"crop": blob}).reset_index().rename({"index": "crop",  "crop": "cropname"}, axis=1)
cal_key = cal_key.applymap(lambda x: x.strip() if isinstance(x, str) else x)

df = fao_key.merge(area_key, how="left")

cal_key.loc[cal_key['cropname']=="Cotton", 'cropname'] = "Seed cotton"
cal_key.loc[cal_key['cropname']=="Oat", 'cropname'] = "Oats"
cal_key.loc[cal_key['cropname']=="Pulses", 'cropname'] = "Pulses nes"
cal_key.loc[cal_key['cropname']=="Sugarbeets", 'cropname'] = "Sugar beet"
cal_key.loc[cal_key['cropname']=="Sunflower", 'cropname'] = "Sunflower seed"

df= df.merge(cal_key, how="left").dropna().reset_index()
df = cal_key.merge(df, how="left")
#df.to_pickle("./data/calendar_fao_cropkey.pkl")

# country names

In [34]:
mylist = [f for f in glob.glob("/Users/caropark/Dropbox (Harvard University)/DataByCountry/*")]
angela_names = pd.DataFrame({'country': np.unique([path.rsplit('/', 1)[1] for path in mylist])})

flags = pd.read_csv("./data/faostat_all_flags.csv")
fao_names = pd.DataFrame(np.unique(flags['Area']))

countries = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
lowres_names = pd.DataFrame(np.unique(countries['name']))

inc_class = pd.read_csv("./data/wb_classification.csv", index_col=None)
inc_class['country']=inc_class['country'].astype(str)
wb_names = pd.DataFrame(np.unique(inc_class['country']))

In [35]:
def find_best_matches(df1, df2, df2_name, top_n):
    matches = []
    df1_column = df1.iloc[:, 0]
    df2_column = df2.iloc[:, 0]
    
    for country in df1_column:
        results = process.extract(country, df2_column, limit=top_n)
        
        if any(score == 100 for _, score, _ in results):
            best_match = next((match for match, score, _ in results if score == 100), None)
            matches.append((country, best_match, 100))
        
        else:
            filtered_results = [(country, match, score) for match, score, _ in results if score > 60]
            
            if filtered_results: 
                matches.extend(filtered_results)
            else:  
                matches.append((country, None, None))
        
    return pd.DataFrame(matches, columns=['country', df2_name, 'sim_score'])


In [36]:
matches_100list = list()
matches_n100list = list()

dataframes = {'fao': fao_names, 'lowres': lowres_names, 'wb': wb_names}

for name, df in dataframes.items():
    matches = find_best_matches(angela_names, df, name, top_n=3)
    m100 = matches[matches['sim_score']==100].drop(columns="sim_score")
    mn100 = matches[matches['sim_score']<100]
    matches_100list.append(m100)
    matches_n100list.append(mn100)


In [37]:
country_key = reduce(lambda x, y: pd.merge(x, y, on = 'country', how="outer"), matches_100list)
country_key = pd.merge(angela_names, country_key, on="country", how="left").reset_index(drop=True)

In [38]:
keep_fao= matches_n100list[0].reset_index(drop=True).iloc[[3,4,14,17, 21], 0:2]

keep_lowres= matches_n100list[1].reset_index(drop=True).iloc[[7,10,13,19,28, 30,37,38,41,47,50,52,67,92,93,95,108,111,115,117,119,127,129,132,135], 0:2]

keep_wb = matches_n100list[2].reset_index(drop=True).iloc[[0,2,5,9,14,18,23,30,34,46,50,55,61,63,68,72,74,77,81,83,86,91,93,95,98,104,106], 0:2]

In [39]:
merged_df = pd.merge(keep_fao, keep_lowres, on='country', how='outer')
fixed = pd.merge(merged_df, keep_wb, on='country', how='outer')
fixed = pd.merge(angela_names, fixed, on="country", how="left").reset_index(drop=True)

In [40]:
country_key = country_key.combine_first(fixed)

In [41]:
fixagain= country_key[country_key.isna().any(axis=1)].reset_index(drop=True)

In [42]:
fixed2 = pd.DataFrame({'country': ["Lao People's Democratic Republic", "China, Taiwan Province of", "Democratic Republic of the Congo", "Democratic People's Republic of Korea", "Iran (Islamic Republic of)", "Venezuela (Bolivarian Republic of)"],
                      'lowres': ["Laos", np.nan, np.nan, np.nan, np.nan, np.nan],
                      'wb': [np.nan, "Taiwan, China", "Congo, Dem. Rep.", "Korea, Dem. People's Rep.", "Iran, Islamic Rep.", "Venezuela, RB"]})
fixed2 = pd.merge(angela_names, fixed2, on='country', how="left").reset_index(drop=True)

In [43]:
country_key = country_key.combine_first(fixed2)

In [44]:
isos = flags[['Area', 'Area Code (ISO3)']].drop_duplicates()
isos.columns=['fao', 'iso_a3']
country_key = country_key.merge(isos, how="left", on="fao")

In [45]:
country_key.to_csv("./data/country_key.csv", index=False)